In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from random import random

In [2]:
#Load ratings csv file

ratings_df = pd.read_csv("ratings_small.csv")#need to try this on full dataset

ratings_df.shape

ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Load Movies metadata csv file

In [3]:
movies_df = pd.read_csv("input/movies_metadata.csv")

movies_df.head()

/home/kadekool/class/4022/env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


Merge the two dataframe to keep only userId, movieId, rating and title data

In [4]:
movies_df = movies_df[movies_df['id'].astype(str).str.isdigit()] # clean (Some movies have not integers as id)
movies_df.id = movies_df.id.astype(np.int64)

type(movies_df.id[0])

ratings_df.movieId.isin(movies_df.id).sum()

ratings_df = pd.merge(ratings_df,movies_df[['title','id']],left_on='movieId',right_on='id')
ratings_df.head()

,userId,movieId,rating,timestamp,title,id
0,1,1371,2.5,1260759135,Rocky III,1371
1,4,1371,4.0,949810302,Rocky III,1371
2,7,1371,3.0,851869160,Rocky III,1371
3,19,1371,4.0,855193404,Rocky III,1371
4,21,1371,3.0,853852263,Rocky III,1371


In [5]:
ratings_df.drop(['timestamp','id'],axis=1,inplace=True)

ratings_df.shape

ratings_df.sample(5)

,userId,movieId,rating,title
31085,472,5991,4.0,The Last Laugh
37867,417,4499,4.5,"Pepi, Luci, Bom"
41780,306,132,3.0,The Rolling Stones: Gimme Shelter
22596,83,6,5.0,Judgment Night
4354,23,508,4.5,Love Actually


Let's see the number of ratings for each movie

Let's see some statistics for the totalRatings

In [7]:
ratings_top = ratings_df

ratings_top.shape

ratings_top.head()

,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
1,4,1371,4.0,Rocky III
2,7,1371,3.0,Rocky III
3,19,1371,4.0,Rocky III
4,21,1371,3.0,Rocky III


Make data consistent by ensuring there are unique entries for [title,userId] pairs

In [8]:
if not ratings_top[ratings_top.duplicated(['userId','title'])].empty:
    ratings_top = ratings_top.drop_duplicates(['userId','title'])

ratings_top.shape

(44845, 4)

Reshape the data using pivot function

In [9]:
df_for_knn = ratings_top.pivot(index='title',columns='userId',values='rating').fillna(0)

In [10]:
df_for_knn.tail(15)

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
title,,,,,,,,,,,,,,,,,,,,,
Zapped Again!,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zardoz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zatoichi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zatôichi's Pilgrimage,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zazie dans le métro,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zodiac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zombie Flesh Eaters,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zombie Holocaust,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Zozo,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
titles = [df_for_knn.index[i] for i in range(len(df_for_knn))]

def neighbors(df, user = None, user_col = None):

    if user is None:
        distances = [np.linalg.norm(df[user_col] - df[u]) for u in df.columns]
        distances_sorted = np.sort(distances)
        nearest = [distances.index(distances_sorted[i]) for i in range(len(distances_sorted))]
        
    else:
        distances = [np.linalg.norm(user - df[u]) for u in df.columns]
        distances_sorted = np.sort(distances)
        nearest = [distances.index(distances_sorted[i]) for i in range(len(distances_sorted))]
        
    return nearest

def find_neighbors(df = df_for_knn, user = None, user_col = 0):
    if user is None:
        return neighbors(df, user_col = user_col)
    else:
        return neighbors(df, user = user)
    
def find_recs(df = df_for_knn, user = None, user_col = 0, k = 1):
    neighbor = []
    
    if user is None:
        neighbor = find_neighbors(df, user_col = user_col)
    else:
        neighbor = find_neighbors(df, user = user)
    
    scores = [(i,score) for i,score in enumerate(neighbor) if user[i] == 0]
    scores.sort(key = lambda x:x[1])
    
    return [titles[scores[i][0]] for i in range(k)]
    
random_user = [int(random()*4+1) if random() >= 0.9 else 0 for _ in range(len(df_for_knn))]

temp = find_recs(user = random_user, k = 10)

print(temp)

['...And God Created Woman', 'Brother Sun, Sister Moon', 'Arthur and the Revenge of Maltazard', 'Dial M for Murder', 'Cherry Blossoms', 'Austin High', 'Blind Man', 'Cabiria', 'Bandyta', 'Bang, Boom, Bang']
